# Listado de funcionarios públicos por institución
Este cuaderno se muestra cómo obtener la lista de nombres de funcionarios públicos disponibles en el portal de transparencia gubernamental de El Salvador (https://www.transparencia.gob.sv/api/v1/officials). Los datos se recaban a partir del 2019-06-01 a manera de ejemplo.

In [ ]:
import requests
from urllib.parse import urlencode
import pandas as pd
import csv

Primero definimos la url base a utilizar para obtener los datos de las instituciones públicas del API (ver ejemplo de cuaderno de días sin actualizar) y los parámetros para hacer la consulta.

In [ ]:
url_instituciones = 'https://www.transparencia.gob.sv/api/v1/institutions.json?'
datos_instituciones = {"start_at":"2010-01-01", "end_at":"2022-07-31", "per_page":1000, "page":1}

Descargamos los datos de todas las instituciones del API. El try ... except sirve para gestionar errores en caso de error. Revisar ejemplos aquí: https://www.w3schools.com/python/python_try_except.asp

In [ ]:
try:
    url = url_instituciones + urlencode(datos_instituciones)
    instituciones = requests.get(url).json()
except:
    print("Error en la obtención de los archivos. Intente de nuevo.")

In [ ]:
datos_funcionarios = {"start_at":"2019-06-01", "end_at":"2022-07-31", "institution_id":1, "per_page":10000, "page":1}
url_funcionarios = 'https://www.transparencia.gob.sv/api/v1/officials.json?'

In [ ]:
#Usaremos esta variable para identificar si estamos en la primera iteración y así colocar los encabezados al archivo CSV
iteracion = 0

Creamos un bucle en el que descargaremos los datos de cada institucion. Luego, crearemos un dataframe con pandas y con ellos se guardarán los datos en un archivo "resultados.csv". Esto puede tardar varios minutos. El archivo estará en la misma carpeta del notebook. Para efectos de prueba, solo se descargan los datos de las primeras 25 instituciones, pero si se remueve la expresión [:25] descargará todo (toma varios minutos). 

In [ ]:
for institucion in instituciones[:25]:
    datos_funcionarios['institution_id'] = institucion['id']
    url = url_funcionarios + urlencode(datos_funcionarios)
    try:
        datos = requests.get(url).json()
    except:
        print("Hubo un error en la obtención de datos.")
    if len(datos) > 0: #Verificamos que la respuesta no esté vacia
        df = pd.DataFrame(datos)
        df.drop(columns = ["curriculum"], inplace=True) #Eliminamos los datos de los CV por ocupar demasiado espacio
    else:
        continue
    if iteracion:
        df.to_csv('resultado.csv', index=False, quoting=csv.QUOTE_ALL, header=False, mode="a")
    else:
        df.to_csv('resultado.csv', index=False, quoting=csv.QUOTE_ALL)
        iteracion += 1
    print("Continuo descargando...")